In [1]:


import pandas as pd
import os
import numpy as np
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation

import seaborn as sns
import matplotlib.pyplot as plt
import datatable as dt

SEED = 123

data = dt.fread('~/jane-street-market-prediction/train.csv').to_pandas()


In [2]:
unique_dates = np.unique(data['date'] )
print(unique_dates)
np.random.shuffle(unique_dates)
print(unique_dates)

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 244 245 24

In [3]:
folds = 5
idx = np.linspace(start = 0, stop = len(unique_dates)-1, num = folds + 1, dtype = int)

In [4]:
mask = data['date'].isin(unique_dates[idx[0]:idx[0+1]] )

In [ ]:
 def create_mlp2(
    num_columns, num_labels, hidden_units, dropout_rates, label_smoothing, learning_rate
):

    inp = tf.keras.layers.Input(shape=(num_columns,))
    x = tf.keras.layers.BatchNormalization()(inp)
    #x = tf.keras.layers.BatchNormalization()(inp)
    x = tf.keras.layers.Dropout(dropout_rates[0])(x) #(x)
    
    for i in range(len(hidden_units)):
        x = tf.keras.layers.Dense(hidden_units[i])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation(tf.keras.activations.relu)(x)
        #x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)
        x = tf.keras.layers.Dropout(dropout_rates[i + 1])(x)
        x = tf.keras.layers.Concatenate(axis=1)([x, inp])
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dense(hidden_units[i])(x)
        x = tf.keras.layers.Dropout(dropout_rates[i + 1])(x)

    x = tf.keras.layers.Dense(num_labels)(x)
    out = tf.keras.layers.Activation("sigmoid")(x)

    model = tf.keras.models.Model(inputs=inp, outputs=out)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=label_smoothing),
        metrics=tf.keras.metrics.AUC(name="AUC"),
    )

    return model

In [ ]:
folds = 5
idx = np.linspace(start = 0, stop = len(unique_dates)-1, num = folds + 1, dtype = int)
for f in range(folds):
    mask = data['date'].isin(unique_dates[idx[f]:idx[f+1]] )
    train = data[~mask]
    val = data[mask]
    
    train.fillna(train.mean(),inplace=True)

    features = [c for c in train.columns if "feature" in c]

    f_mean = np.mean(train[features[1:]].values,axis=0)

    X_train = train.loc[:, train.columns.str.contains('feature')]
    y_train = (train.loc[:, 'action'])
    

    batch_size = 5000
    hidden_units = [260, 260, 130]
    dropout_rates = [0.2, 0.2, 0.2, 0.2]
    label_smoothing = 1e-2
    learning_rate = 1e-3

    clf = create_mlp2(
        len(features), 1, hidden_units, dropout_rates, label_smoothing, learning_rate
        )

    clf.fit(X_train, y_train, epochs=200, batch_size=20000)

    opt_th = 0.5000
    
    

In [ ]:
pred_df = train[['date', 'resp','weight']]
pred_df['actual'] = ((train['resp'].values) > 0).astype(int)

In [ ]:
preds = clf(X_train.values, training=False)

In [ ]:
from math import sqrt

def utility_score(df,action_vec):
    """Calculate utility score of a dataframe according to formulas defined at
    https://www.kaggle.com/c/jane-street-market-prediction/overview/evaluation
    """

    p = df['weight']  * df['resp'] * action_vec
    df['p'] = p
    p_i = df.set_index('date')['p'].groupby('date').sum()
    t = (p_i.sum() / sqrt((p_i**2).sum())) * (sqrt(250 / p_i.index.size))
    return min(max(t, 0), 6) * p_i.sum()

In [ ]:
pred_df['preds'] = preds.numpy()

In [ ]:
pred_df['action'] = (pred_df['preds'] >= 0.5) * 1

In [ ]:
utility_score(pred_df, pred_df['action']) / utility_score(pred_df, pred_df['actual'])
